In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import os
import time
import datetime as dt
from collections import defaultdict



In [2]:
data = pd.read_csv("C:/Users/Nandini Jain/Desktop/MSEcoSem2/Data_CohortAnalysis.csv")

In [3]:
#changing invoicedate from object to datetime format, using 'mixed' here since some values are in "%m/%d/%Y %H:%M"
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'],format = 'mixed')

In [4]:
data.head

<bound method NDFrame.head of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

               InvoiceDate  UnitPrice  Custom

In [5]:
data['CohortMonth'] = data.groupby('CustomerID')['InvoiceDate'].transform('min').dt.to_period('M')
data['TransactionMonth'] = data['InvoiceDate'].dt.to_period('M')


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         541909 non-null  object        
 1   StockCode         541909 non-null  object        
 2   Description       540455 non-null  object        
 3   Quantity          541909 non-null  int64         
 4   InvoiceDate       541909 non-null  datetime64[ns]
 5   UnitPrice         541909 non-null  float64       
 6   CustomerID        406829 non-null  float64       
 7   Country           541909 non-null  object        
 8   CohortMonth       406829 non-null  period[M]     
 9   TransactionMonth  541909 non-null  period[M]     
dtypes: datetime64[ns](1), float64(2), int64(1), object(4), period[M](2)
memory usage: 41.3+ MB


In [7]:
data = data.dropna(subset=['CustomerID'])

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         406829 non-null  object        
 1   StockCode         406829 non-null  object        
 2   Description       406829 non-null  object        
 3   Quantity          406829 non-null  int64         
 4   InvoiceDate       406829 non-null  datetime64[ns]
 5   UnitPrice         406829 non-null  float64       
 6   CustomerID        406829 non-null  float64       
 7   Country           406829 non-null  object        
 8   CohortMonth       406829 non-null  period[M]     
 9   TransactionMonth  406829 non-null  period[M]     
dtypes: datetime64[ns](1), float64(2), int64(1), object(4), period[M](2)
memory usage: 34.1+ MB


In [9]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

transaction_year, transaction_month,_ = get_date_int(data, 'TransactionMonth')
cohort_year, cohort_month,_ = get_date_int(data, 'CohortMonth')

In [12]:
years_diff = transaction_year - cohort_year
months_diff = transaction_month - cohort_month
data.loc['CohortIndex'] = years_diff * 12 + months_diff + 1
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CohortMonth,TransactionMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12,2010-12
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12,2010-12


In [13]:
grouping_count = data.groupby(['CohortMonth', 'CohortIndex'])
cohort_data = grouping_count['CustomerID'].apply(pd.Series.nunique)
cohort_data = cohort_data.reset_index()
cohort_counts = cohort_data.pivot(index='CohortMonth',
                                  columns='CohortIndex',
                                  values='CustomerID')
print(cohort_counts.head())

KeyError: 'CohortIndex'

In [ ]:
# --Calculate Retention Rate--
cohort_sizes = cohort_counts.iloc[:,0]
retention = cohort_counts.divide(cohort_sizes, axis=0)
retention.round(3) * 100
retention.index = retention.index.strftime('%m-%Y')

In [ ]:
# --Calculate Average Quantity--
grouping_qty = data.groupby(['CohortMonth', 'CohortIndex'])
cohort_data_qty = grouping_qty['Quantity'].mean()
cohort_data_qty = cohort_data_qty.reset_index()
average_quantity = cohort_data_qty.pivot(index='CohortMonth',
                                     columns='CohortIndex',
                                     values='Quantity')
average_quantity.index = average_quantity.index.strftime('%m-%Y')
average_quantity

In [ ]:
# --Calculate Average Price--
grouping_price = data.groupby(['CohortMonth', 'CohortIndex'])
cohort_data_price = grouping_price['UnitPrice'].mean()
cohort_data_price = cohort_data_price.reset_index()
average_price = cohort_data_price.pivot(index='CohortMonth',
                                     columns='CohortIndex',
                                     values='UnitPrice')
average_price.index = average_price.index.strftime('%m-%Y')
average_price

In [ ]:
# Plot retention rates
plt.figure(figsize=(10, 8))
plt.title('Retention rates')
sns.heatmap(data = retention, annot = True, fmt = '.0%',vmin = 0.0,vmax = 0.5,cmap = 'BuGn')
plt.show()

In [ ]:
# Plot average quantity
plt.figure(figsize=(10, 8))
plt.title('Average Quantity')
sns.heatmap(data = average_quantity, annot=True, cmap='Blues')
plt.show()

In [ ]:
# Plot average price
plt.figure(figsize=(10, 8))
plt.title('Average Price')
sns.heatmap(data = average_price, annot=True, cmap='Blues')
plt.show()

In [ ]:
# Grouping by CohortMonth and Item Description to get purchase frequency
cohort_item_prefs = data.groupby(['CohortMonth', 'StockCode'])['Quantity'].sum().reset_index()
# Get top products for each cohort
top_items_per_cohort = cohort_item_prefs.sort_values(['CohortMonth', 'Quantity'], ascending=[True, False]).groupby('CohortMonth').head(5)


top_items_per_cohort